In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from fastai.text import * 


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from langdetect import detect

In [ ]:
!pip install googletrans

In [ ]:
from googletrans import Translator



* Thank you to this kernel https://www.kaggle.com/xhlulu/cord-19-eda-parse-json-and-generate-clean-csv that provided the clean csv files for analysis.
* Thank you to this kernel that provided the keywords for COVID-19 papers only https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles



First we load the data inside each of the csv files.


In [ ]:
path = '/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/'

In [ ]:
pmc_custom_license_df = pd.read_csv(path + "clean_pmc.csv")
noncomm_use_df = pd.read_csv(path + "clean_noncomm_use.csv")
comm_use_df = pd.read_csv(path + "clean_comm_use.csv")
biorxiv = pd.read_csv(path + "biorxiv_clean.csv")

In [ ]:
data = pd.concat([pmc_custom_license_df, noncomm_use_df, comm_use_df, biorxiv], axis =0)

We subset the dataset to have the paper id and text of the document only

In [ ]:
covid_papers = data[data.text.str.contains('COVID-19|SARS-CoV-2|2019-nCov|SARS Coronavirus 2|2019 Novel Coronavirus')][['paper_id', 'text']]

In [ ]:
covid_papers = data[['paper_id', 'text']]

In [ ]:
data = covid_papers

To detect the language of the papers we use the function detect from the langdetect library

In [ ]:
data['language'] = data['text'].map(detect)

In [ ]:
data.shape

In [ ]:
data['language'].value_counts()

Four papers are written in Spanish.

In [ ]:
indices_to_translate = data[data['language']=='es']['text'].index

In [ ]:
translator = Translator()

In [ ]:
#3,900 character limit for google translate

In [ ]:
for i in indices_to_translate:

    if (len(data.loc[i, 'text']) > 3900):
        data.loc[i, 'text'] = data.loc[i, 'text'][:3900]
    paper = translator.translate(data.loc[i, 'text'])
    data.loc[i, 'text'] = paper.text


In [ ]:
data.loc[indices_to_translate, 'text']

The papers are now translated

In [ ]:
data.to_csv('data_covid19_papers.csv')

In [ ]:
data.shape()

# Language model data



We train on ~90% of the data and validate on the other ~10%. The function TextLMDataBunch creates a TextDataBunch suitable for training a language model.


In [ ]:
train, validation = train_test_split(data, test_size=0.2, random_state=42, shuffle= True)

In [ ]:
data_lm = TextLMDataBunch.from_df('.',  train, validation,
                                  label_cols='text')

We save the results of this step so we don't have to repeat them in the future and save a couple of minutes

In [ ]:
data_lm.save('data_lm_export.pkl')

In [ ]:
data_lm = load_data('.', 'data_lm_export.pkl')

In [ ]:
learn = language_model_learner(data_lm,  arch = AWD_LSTM, pretrained = True, drop_mult=0.7)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

First we train the heads for 4 epochs

In [ ]:
learn.fit_one_cycle(4, 2e-2) 

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(1e-4, 1e-3))

In [ ]:
learn.save_encoder('ft_enc')

The language model achieves ~44% accuracy in predicting the next word. This result isn't great, but we will ask the questions to the language model.


# Inference part

Using the fastai function learn.predict we can ask the questions to the language model. We set the temperature from 0.05 to 0.1 to make the results less random (default for the function is temperature= 1 which makes random predictions)


In [ ]:
np.random.seed(42)

In [ ]:
learn.predict("risks factors", n_words=20, temperature=0.1)

In [ ]:
learn.predict("smoking", n_words=100, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("pre-existing pulmonary disease", n_words=50, no_unk=True, temperature=0.05)

In [ ]:
learn.predict("Co-infections", n_words=80, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("comorbidities", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Neonates", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("pregnant women", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Socio-economic factors", n_words=100, no_unk=True, temperature=0.1)  

In [ ]:
learn.predict("behavioral factors", n_words=80, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Transmission dynamics of the virus", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("basic reproductive number", n_words=80, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("incubation period", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("serial interval", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("environmental factors", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("modes of transmission", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("Severity of disease", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("risk of fatality among symptomatic hospitalized patients", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("high-risk patient", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Susceptibility of populations", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Public health mitigation measures", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("What do we know about COVID-19 risk factors?", n_words=100, no_unk=True, temperature=0.1)

The responses are not that great, some make sense, some do not. No new information revealed. We will try to cluster the papers using the embeddings for the language model to see if we can get better inference.